In [2]:
import networkx
import networkx as nx 
import numpy as np
import random
import scipy as sp

import timeit
import cProfile
import pstats
from numba import njit

In [3]:
%run '/Users/janthiel/Desktop/Math Diss/main copy.ipynb'

In [3]:
def generate_graph(n,k):
    # Create a random directed graph
    G = nx.fast_gnp_random_graph(n, 0.7, directed=True)


    '''# Split the nodes into k subsets
    nodes = list(G.nodes())
    random.shuffle(nodes)
    subsets = [set(nodes[i*n//k:(i+1)*n//k]) for i in range(k)]

    # Remove all edges between different subsets
    for u, v in list(G.edges()):
        for i in range(k):
            if u in subsets[i]:
                for j in range(i+1, k):
                    if v in subsets[j]:
                        G.remove_edge(u, v)
                        break

    # Ensure every node has outdegree of at least 1
    for node in G.nodes():
        if G.out_degree(node) == 0:
            # Choose a random node to connect to, excluding the current node
            other_nodes = list(set(G.nodes()) - {node})
            G.add_edge(node, random.choice(other_nodes))'''

    # Now, G is a random directed graph that is not strongly connected but every node has outdegree of at least 1

    G = nx.stochastic_graph(G)
    kc_A = nx.adjacency_matrix(G).todense().T
    return G, kc_A

In [4]:
generate_graph(1000,10)

KeyboardInterrupt: 

In [5]:
def stochastic(A):
    for i in range(np.shape(A)[0]):
        A[:, i] = A[:, i]/sum(A[:, i])
    return A

def find_descendants(graph):
    sorted_nodes = list(nx.topological_sort(graph))
    descendants = {node: set() for node in sorted_nodes}
    for node in reversed(sorted_nodes):  # start from the end of the list
        for neighbor in graph.successors(node):  # successors are the children of a node in a directed graph
            descendants[node].add(neighbor)
            descendants[node] |= descendants[neighbor]  # add descendants of the neighbor
    return descendants 

def find_ancestors(graph):
    sorted_nodes = list(nx.topological_sort(graph))
    ancestors = {node: set() for node in sorted_nodes}
    for node in sorted_nodes:  # start from the beginning of the list
        ancestors[node].add(node)
        for neighbor in graph.predecessors(node):  # predecessors are the parents of a node in a directed graph
            ancestors[node].add(neighbor)
            ancestors[node] |= ancestors[neighbor]  # add ancestors of the neighbor
    return ancestors 


def N(Graph, A, alpha):
    n = np.shape(A)[0]
    v = pagerank_power(A.T, p=alpha)
    v = ((n*v - ((1-alpha)/alpha))/n).reshape(-1,)
    C = list(networkx.strongly_connected_components(Graph)) 
    M = np.zeros((n,n))

    G1 = networkx.condensation(Graph, C)

    descendants = find_descendants(G1)
    descendants = {k: u for k, u in descendants.items() if u}

    for k, u in reversed(descendants.items()):
        for p in C[k]:
            for element in u:
                for s in C[element]:
                    M[s,p] = v[s]

    for t in range(len(C)):
        for i in tuple(C[t]):
            M[i, tuple(C[t])] = v[i]

 
    M += ((1-alpha)/alpha)*np.eye(n)
    M = (alpha+0.0175)*stochastic(M)+(1-alpha-0.0175)*A
    return M

In [6]:
def stochastic(A):
    for i in range(np.shape(A)[0]):
        A[:, i] = A[:, i]/sum(A[:, i])
    return A

def find_descendants(graph):
    sorted_nodes = list(nx.topological_sort(graph))
    descendants = {node: set() for node in sorted_nodes}
    for node in reversed(sorted_nodes):  # start from the end of the list
        for neighbor in graph.successors(node):  # successors are the children of a node in a directed graph
            descendants[node].add(neighbor)
            descendants[node] |= descendants[neighbor]  # add descendants of the neighbor
    return descendants 

def find_ancestors(graph):
    sorted_nodes = list(nx.topological_sort(graph))
    ancestors = {node: set() for node in sorted_nodes}
    for node in sorted_nodes:  # start from the beginning of the list
        ancestors[node].add(node)
        for neighbor in graph.predecessors(node):  # predecessors are the parents of a node in a directed graph
            ancestors[node].add(neighbor)
            ancestors[node] |= ancestors[neighbor]  # add ancestors of the neighbor
    return ancestors 


def L(Graph, A, alpha):
    n = np.shape(A)[0]
    v = pagerank_power(A.T, p=alpha)
    v = ((n*v - ((1-alpha)/alpha))/n).reshape(-1,)
    C = list(networkx.strongly_connected_components(Graph)) 
    M = np.zeros((n,n))

    G1 = networkx.condensation(Graph, C)

    descendants = find_descendants(G1)
    descendants = {k: u for k, u in descendants.items() if u}

    for k, u in reversed(descendants.items()):
        for p in C[k]:
            for element in u:
                for s in C[element]:
                    M[s,p] = v[s]

    for t in range(len(C)):
        for i in tuple(C[t]):
            M[i, tuple(C[t])] = v[i]

 
    M += ((1-alpha)/alpha)*np.eye(n)
    M = (alpha+0.0175)*(M)+(1-alpha-0.0175)*A
    return M

In [7]:
import numpy as np
import networkx as nx
from scipy.sparse import coo_matrix, csr_matrix, eye

def find_all_ancestors(G, C, G1):
    # Initialize an empty dictionary to store the ancestors of each node
    ancestors = {node: set() for node in G.nodes()}
    # Iterate over the nodes in the topological order of the condensed graph
    for node in (nx.topological_sort(G1)):
        # If the node has predecessors in the condensed graph
        if G1.in_degree(node) > 0:
            # The ancestors of the node are the union of its own ancestors and its predecessors' ancestors
            for pred in G1.predecessors(node):
                ancestors[node] = ancestors[node].union(C[pred])
        # Add the ancestors to the nodes in the original graph that correspond to the current node in the condensed graph
        for original_node in C[node]:
            ancestors[original_node] = ancestors[node].union(C[node])
    # Create u as a list of sets, in the same order as the nodes in the original graph
    u = [ancestors[node] for node in G.nodes()]
    return u



def T(Graph, alpha):
    n = len(Graph)
    A = nx.to_scipy_sparse_array(Graph, format='csr')  # Use CSR format directly
    v = pagerank_power(A, p=alpha)
    m = (((n*v) - ((1-alpha)/alpha))).reshape(-1,) #v = n * v - (1 - alpha) / alpha
    C = list(nx.strongly_connected_components(Graph))
    condensed_graph = nx.condensation(Graph, C)

    u = find_all_ancestors(Graph, C, condensed_graph)
    row_indices = []
    col_indices = []
    data = [] #try creating a sparse matrix directly (putting 1 for every ancestor and then multiply each row by v[k]/q)

    for j, ancestors_set in enumerate(u):
        t = len(ancestors_set)
        row_indices.extend([j] * t)
        col_indices.extend(list(ancestors_set))
        data.extend([m[j] / t] * t)

    M = coo_matrix((data, (row_indices, col_indices)), shape=(n, n))

    eye_matrix = eye(n, format='coo')
    M += ((1 - alpha) / alpha) * eye_matrix
    M = (alpha + 0.0175) * M + (1 - alpha - 0.0175) * A.T

    return csr_matrix(M)  # Convert to CSR format for efficient arithmetic operations


In [8]:
def Fast(Graph, alpha):
    n = len(Graph)
    A = nx.to_scipy_sparse_array(Graph, format='csr')  # Use CSR format directly
    v = pagerank_power(A, p=alpha)
    m = 1/n*(n*v-(1-alpha)/alpha)
    C = list(nx.strongly_connected_components(Graph))
    condensed_graph = nx.condensation(Graph, C)

    u = find_all_ancestors1(Graph, C, condensed_graph)
    
    row_indices = []
    col_indices = []
    data = [] #try creating a sparse matrix directly (putting 1 for every ancestor and then multiply each row by v[k]/q)

    for j, ancestors_set in enumerate(u):
    #for j, ancestors_set in u.items():
        t = len(ancestors_set)
        #m = 1/n*(n*v-(1-alpha)/alpha) #we can make this line a lot more efficient by calculating it at the beginning 
        row_indices.extend([j] * t)
        col_indices.extend(list(ancestors_set))
        data.extend([m[j]] * t)

    #print('we are done loading the matrix')
    M = coo_matrix((data, (row_indices, col_indices)), shape=(n, n))
    print(f"Fast, there are {M.nnz} ancestors")

    eye_matrix = eye(n, format='coo')
    M += ((1 - alpha) / alpha) * eye_matrix
    M = (alpha + 0.0175) * M + (1 - alpha - 0.0175) * A.T

    return csr_matrix(M) 

In [9]:
import numpy as np
import networkx as nx
from scipy.sparse import coo_matrix, csr_matrix, eye

import networkx as nx

import networkx as nx

def flatten(xss):
    return [x for xs in xss for x in xs]

def find_all_ancestors1(G, C, G1):
    # Initialize a list to store the ancestors of each node
    ancestors = [set() for _ in range(len(G.nodes()))]
    predecessors = {node: set() for node in G1.nodes()}

    for node in nx.topological_sort(G1):
        for pred in G1.predecessors(node):
            predecessors[node].update(C[pred])#, predecessors[pred])
        components = C[node]
        for original_node in components:
            ancestors[original_node] = components | predecessors[node]

    u = [list(ancestors[node]) for node in G.nodes()]
    print(len(u))


def find_all_ancestors2(G, C, G1):
    # Initialize a list to store the ancestors of each node
    ancestors = [[] for _ in range(len(G.nodes()))]
    predecessors = [[] for _ in range(len(G1.nodes()))]
    for node in nx.topological_sort(G1):
        L = list(G1.predecessors(node))
        #temp1 = [(list(C[pred])) for pred in L]
        temp2 = [predecessors[pred] for pred in L]
        predecessors[node] = set(flatten(temp2)) #+ flatten(temp1))
        components = list(C[node])
        for original_node in components:
            ancestors[original_node] = components + list(predecessors[node])

    return ancestors


def find_all_ancestors3(G, C, G1):
    # Initialize a list to store the ancestors of each node
    ancestors = [[] for _ in range(len(G.nodes()))]
    predecessors = {node: set([node]) for node in G1.nodes()}

    for node in nx.topological_sort(G1):
        for pred in G1.predecessors(node):
            predecessors[node].update(predecessors[pred])
        components = list(C[node])
        for original_node in components:
            ancestors[original_node] = flatten([list(C[k]) for k in predecessors[node]])
    return {ancestors}

def find_all_ancestors4(G, C, G1):
    # Initialize a list to store the ancestors of each node
    ancestors = [[] for _ in range(len(G.nodes()))]
    predecessors = {node: set(C[node]) for node in G1.nodes()}
    for node in nx.topological_sort(G1):
        for pred in G1.predecessors(node):
            predecessors[node].update(predecessors[pred])
        for original_node in C[node]:
            ancestors[original_node] = predecessors[node]
    u = [list(ancestors[node]) for node in G.nodes()]
    return u 

In [10]:
'''if networkx.is_strongly_connected(Graph):
        print('strongly connected Graph')
        A = nx.to_numpy_array(Graph)
        v = pagerank_power(A, p=0.85)
        m = 1/n*(n*v-(1-0.85)/0.85)[:, np.newaxis]
        M = np.tile(m, (1, n))
        M += ((1 - alpha) / alpha) * np.eye(n)
        M = (alpha + 0.0175) * M + (1 - alpha - 0.0175) * A.T
        return M'''

"if networkx.is_strongly_connected(Graph):\n        print('strongly connected Graph')\n        A = nx.to_numpy_array(Graph)\n        v = pagerank_power(A, p=0.85)\n        m = 1/n*(n*v-(1-0.85)/0.85)[:, np.newaxis]\n        M = np.tile(m, (1, n))\n        M += ((1 - alpha) / alpha) * np.eye(n)\n        M = (alpha + 0.0175) * M + (1 - alpha - 0.0175) * A.T\n        return M"

In [16]:

def T(Graph, alpha):
    n = len(Graph)
    
    A = nx.to_scipy_sparse_array(Graph, format='coo')  # Use coo format directly
    v = pagerank_power(A, p=alpha)
    m = 1/n*(n*v-(1-alpha)/alpha)
    
    C = list(nx.strongly_connected_components(Graph))
    condensed_graph = nx.condensation(Graph, C) # This has time complexity O(V+E)

    u = find_all_ancestors5_optimized(Graph, C, condensed_graph) #the maximum total number of ancestors is n * (s-1)/2 
                                                                #because every every node is part of one SCC, every SCC has on average s(s-1)/(2n) ancestors and every SCC has on average n/s nodes
    row_indices = []                                                                                                                           #^not sure this is correct, shoudln't it be 2S? 
    col_indices = []                                                                                                                           # I'm sure that would have been my first instict though as then the terms cancel
    data = [] #try creating a sparse matrix directly (putting 1 for every ancestor and then multiply each row by v[k]/q)                       # And I remember being disappointed that they don't. 
                                                                                                                                                # The mistake is that we are not counting the nodes that i connects to within its own SCC.
    
    #for j, ancestors_set in enumerate(u):#there are at most n * (s-1)/2 nonzero values so the rest of the algorithm has O(n * (s-1)/2)
    for j, ancestors_set in u.items(): 
        t = len(ancestors_set)
        row_indices.extend([j] * t)
        col_indices.extend(list(ancestors_set))
        data.extend([m[j]] * t)

    #print('we are done loading the matrix')
    M = coo_matrix((data, (row_indices, col_indices)), shape=(n, n))
    print(f"Fast, there are {M.nnz} ancestors")

    eye_matrix = eye(n, format='coo')
    M += ((1 - alpha) / alpha) * eye_matrix
    M = (alpha + 0.0175) * M + (1 - alpha - 0.0175) * A.T

    return M #csr_matrix(M)  # Convert to CSR format for efficient arithmetic operations

In [12]:
def find_all_ancestors4_optimized(G, C, G1):
    # Initialize a dictionary to store the ancestors of each node
    ancestors = {node: set() for node in G.nodes}
    predecessors = {node: set(C[node]) for node in G1.nodes()}

    for node in nx.topological_sort(G1):
        for pred in G1.predecessors(node):
            predecessors[node].update(predecessors[pred])
        for original_node in C[node]:
            ancestors[original_node] = predecessors[node]
    
    # Convert sets to lists at the end
    #u = [list(ancestors[node]) for node in G.nodes()]
    return ancestors


In [13]:
import copy

In [17]:

def find_all_ancestors5_optimized(G, C, G1):
    # Initialize a dictionary to store the ancestors of each node
    ancestors = {node: set() for node in G.nodes}
    predecessors = copy.deepcopy(C)#[set() for node in G1.nodes()]#[set(C[node]) for node in G1.nodes()] # 
    
    for node in nx.topological_sort(G1): #s G1 is DAG so it has s nodes and s(s-1)/2 edges so it has time complexity O(s+s(s-1)/2)
        for pred in G1.predecessors(node): #The total number of predecessors (node x pred(node)) is s(s-1)/2.
            predecessors[node].update(predecessors[pred])
        for original_node in C[node]: #(on average n/s) 
            ancestors[original_node] = predecessors[node]
    
    #u = [list(ancestors[node]) for node in G.nodes()]
    #return u
    return ancestors #the time complexity is O(s+s(s-1)/2) + O(s(s-1)/2 + s*n/s) = O(s+s(s-1)/2) + O(s(s-1)/2 + n)


In [15]:
G, kc_A = generate_graph(5,5)

In [23]:
D = list(networkx.strongly_connected_components(G)) 
G1 = networkx.condensation(G, D)
branches = networkx.dag_to_branching(G1)
sorted_nodes = list(networkx.topological_sort(G1))

In [37]:
#B = M(G, np.asarray(kc_A), 0.85)
B = T(G, 0.85)
B.toarray()

Fast, there are 21 ancestors


array([[ 0.2637829 ,  0.3962829 ,  0.30794956,  0.30794956,  0.3300329 ],
       [ 0.12851766,  0.12851766,  0.17268433,  0.17268433,  0.19476766],
       [ 0.        ,  0.        , -0.00459265,  0.        ,  0.        ],
       [ 0.37814769,  0.24564769,  0.24564769,  0.24564769,  0.24564769],
       [ 0.08105616,  0.08105616,  0.12522283,  0.12522283,  0.08105616]])

In [205]:
C = L(G, kc_A, 0.85)
C


array([[0.16281557, 0.00972733, 0.01201182, ..., 0.01213642, 0.00972733,
        0.00972733],
       [0.01147757, 0.16206581, 0.01126205, ..., 0.00897757, 0.00897757,
        0.01122333],
       [0.01147434, 0.0112989 , 0.16206258, ..., 0.01138343, 0.00897434,
        0.00897434],
       ...,
       [0.01130052, 0.01112508, 0.00880052, ..., 0.16188875, 0.01188191,
        0.01104628],
       [0.01229615, 0.01212071, 0.01208063, ..., 0.01220524, 0.16288438,
        0.00979615],
       [0.01032985, 0.01032985, 0.01261434, ..., 0.01273894, 0.01032985,
        0.16341809]])

In [71]:
condition = True
while condition == True:
    n = random.randint(5, 10)
    k = random.randint(5, n)
    G, kc_A = generate_graph(n,k)
    B = T(G,0.85).toarray()
    C = L(G, kc_A, 0.85)
    if np.max((B - C))>0.0005:
        condition = False

In [46]:
l = []
m = []
for i in range(50):
    n = random.randint(10, 100)
    k = 1 #random.randint(10, n)
    F, kc_B = generate_graph(n,k)
    A = T(F,0.85).toarray()
    #B = Fast(F,0.85).toarray()
    #B = L(F, kc_B, 0.85)
    #B = N(F, kc_B, 0.85)
    B = fast_powerrecommend_Matrix2(kc_B, 0.15)#fast_powerrecommend_Matrix2(kc_A, 0.15)
    #U = (sp.stats.rankdata(A, axis=0) - sp.stats.rankdata(B, axis=0))
    #U = U[U>0]#
    l.append(sp.stats.kendalltau(A,B)[0])

Fast, there are 1600 ancestors
Fast, there are 1225 ancestors
Fast, there are 7225 ancestors
Fast, there are 8100 ancestors
Fast, there are 9025 ancestors
Fast, there are 7921 ancestors
Fast, there are 289 ancestors
Fast, there are 961 ancestors
Fast, there are 8836 ancestors
Fast, there are 9025 ancestors
Fast, there are 2025 ancestors
Fast, there are 8836 ancestors
Fast, there are 225 ancestors
Fast, there are 1089 ancestors
Fast, there are 144 ancestors
Fast, there are 400 ancestors
Fast, there are 225 ancestors
Fast, there are 3025 ancestors
Fast, there are 5476 ancestors
Fast, there are 3721 ancestors
Fast, there are 6889 ancestors
Fast, there are 625 ancestors
Fast, there are 961 ancestors
Fast, there are 7921 ancestors
Fast, there are 7396 ancestors
Fast, there are 3721 ancestors
Fast, there are 1296 ancestors
Fast, there are 1444 ancestors
Fast, there are 1600 ancestors
Fast, there are 9409 ancestors
Fast, there are 196 ancestors
Fast, there are 2304 ancestors
Fast, there are 1

In [47]:
sum(l)/50

In [ ]:
A = T(F,0.85)

strongly connected Graph


Fast, there are 2704 ancestors
Fast, there are 3249 ancestors
Fast, there are 441 ancestors


Fast, there are 2209 ancestors


In [ ]:
np.shape(A)

In [192]:
np.shape(B)

In [18]:
import networkx as nx
import json
import pandas as pd
edges_path = '/Users/janthiel/Downloads/facebook_large/musae_facebook_edges.csv'
targets_path = '/Users/janthiel/Downloads/facebook_large/musae_facebook_target.csv'
features_path = '/Users/janthiel/Downloads/facebook_large/musae_facebook_features.json'
edges = pd.read_csv(edges_path)
targets = pd.read_csv(targets_path)
targets.index = targets.id
with open(features_path) as json_data:
    features = json.load(json_data)
fb_graph = networkx.convert_matrix.from_pandas_edgelist(edges, "id_1", "id_2", create_using=networkx.MultiDiGraph())
fb_graph = networkx.stochastic_graph(fb_graph)
fb_A = networkx.to_scipy_sparse_array(fb_graph, format='lil')

In [137]:
F_ = Fast(fb_graph, 0.85)

Fast, there are 193293 ancestors


In [107]:
A = np.array([[0,0.4, 0.3],[0.2,0, 0.7],[0.8,0.6,0]])

In [103]:
A

array([[0. , 0.4, 0.3],
       [0.2, 0. , 0.7],
       [0.8, 0.6, 0. ]])

In [111]:
for i in range(1):
    A = fast_powerrecommend_Matrix2(A, 0.15)
A

array([[0.38758305, 0.21144561, 0.21141648],
       [0.27697225, 0.45316794, 0.27711789],
       [0.3354447 , 0.33538645, 0.51146562]])

In [97]:
A - 0.15/0.85*np.eye(3)

array([[0.21134825, 0.21134826, 0.21134826],
       [0.27693909, 0.27693909, 0.2769391 ],
       [0.33524207, 0.33524207, 0.33524205]])

In [72]:
def iterate(p):
    p = (1 - 0.85)/(1-p*0.85)
    return p

In [84]:
p = o
for i in range(200):
    p = iterate(p)
p

In [71]:
0.15+(0.15**2)*0.85 + (0.15**3)*0.85**2 + (0.15**4)*0.85**3 +  (0.15**5)*0.85**4 + (0.15**6)*0.85**5

In [109]:
F_ = F_.toarray()

In [19]:
T_ = T(fb_graph, 0.85)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'pagerank_power': Cannot determine Numba type of <class 'function'>

File "../../../../var/folders/5f/tb__hdld7xq1_6ks4_y00w8h0000gn/T/ipykernel_13146/1658605919.py", line 6:
<source missing, REPL/exec in use?> 

This error may have been caused by the following argument(s):
- argument 0: Cannot determine Numba type of <class 'networkx.classes.multidigraph.MultiDiGraph'>


In [110]:
T_ = T_ .toarray()

In [125]:
sp.stats.kendalltau(F_[:,:10000],T_[:,:10000])

SignificanceResult(statistic=0.08848169138302224, pvalue=0.0)

In [105]:
slow = pstats.Stats('restats')
slow.sort_stats('cumulative').print_stats()

Sat Mar  2 11:43:49 2024    restats

         3814174 function calls (3701813 primitive calls) in 36.193 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   36.193   36.193 {built-in method builtins.exec}
        1    0.573    0.573   36.193   36.193 <string>:1(<module>)
        1    1.339    1.339   35.619   35.619 /var/folders/5f/tb__hdld7xq1_6ks4_y00w8h0000gn/T/ipykernel_8988/2148921207.py:1(T)
        5    0.000    0.000   15.807    3.161 /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/sparse/_coo.py:132(__init__)
        1    0.202    0.202   11.703   11.703 /var/folders/5f/tb__hdld7xq1_6ks4_y00w8h0000gn/T/ipykernel_8988/3442008535.py:1(find_all_ancestors5_optimized)
      200    9.240    0.046    9.240    0.046 {built-in method numpy.asarray}
       54    0.019    0.000    7.321    0.136 /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/

In [104]:
cProfile.run('T(fb_graph, 0.85)', 'restats')

In [93]:
fast = pstats.Stats('restats')
fast.sort_stats('cumulative').print_stats()

Sat Mar  2 11:37:44 2024    restats

         3072661 function calls (3050179 primitive calls) in 2.743 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.743    2.743 {built-in method builtins.exec}
        1    0.005    0.005    2.743    2.743 <string>:1(<module>)
        1    0.048    0.048    2.738    2.738 /var/folders/5f/tb__hdld7xq1_6ks4_y00w8h0000gn/T/ipykernel_8988/3503103258.py:1(Fast)
        5    0.009    0.002    1.703    0.341 /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/networkx/utils/backends.py:409(__call__)
        1    0.437    0.437    0.870    0.870 /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/networkx/convert_matrix.py:486(to_scipy_sparse_array)
        1    0.000    0.000    0.825    0.825 <class 'networkx.utils.decorators.argmap'> compilation 20:1(argmap_condensation_17)
        1    0.010    0.010  

In [92]:
cProfile.run('Fast(fb_graph, 0.85)', 'restats')

In [243]:
G = fb_graph

In [244]:
C = list(nx.strongly_connected_components(G))
G1 = nx.condensation(G, C)
L = networkx.topological_sort(G1)

In [170]:
#ancestors = {node: set() for node in G.nodes}
predecessors = [set(C[node]) for node in G1.nodes()]#copy.deepcopy(C) #[set(C[node]) for node in G1.nodes()] #

In [264]:
cProfile.run('find_all_ancestors5_optimized(G, C, G1)')

         1155805 function calls (1065925 primitive calls) in 10.737 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.176    0.176   10.009   10.009 3445560071.py:1(find_all_ancestors5_optimized)
        1    0.027    0.027    0.027    0.027 3445560071.py:3(<dictcomp>)
        1    0.728    0.728   10.737   10.737 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 backends.py:409(__call__)
  67411/1    2.933    0.000    3.138    3.138 copy.py:128(deepcopy)
    22470    0.003    0.000    0.003    0.000 copy.py:182(_deepcopy_atomic)
  22471/1    0.043    0.000    3.138    3.138 copy.py:201(_deepcopy_list)
    44941    0.027    0.000    0.037    0.000 copy.py:243(_keep_alive)
    22470    0.039    0.000    2.986    0.000 copy.py:259(_reconstruct)
    44940    0.013    0.000    2.944    0.000 copy.py:264(<genexpr>)
      135    0.076    0.001    0.124    0.001 dag.py:166(topological_generations)
   

In [249]:
cProfile.run('find_all_ancestors5_optimized(G, C, G1)')

         940042 function calls (850162 primitive calls) in 0.517 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.023    0.023    0.514    0.514 1887408157.py:1(find_all_ancestors5_optimized)
        1    0.007    0.007    0.007    0.007 1887408157.py:3(<dictcomp>)
        1    0.002    0.002    0.517    0.517 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 backends.py:409(__call__)
  67411/1    0.156    0.000    0.370    0.370 copy.py:128(deepcopy)
    22470    0.003    0.000    0.003    0.000 copy.py:182(_deepcopy_atomic)
  22471/1    0.041    0.000    0.370    0.370 copy.py:201(_deepcopy_list)
    44941    0.032    0.000    0.043    0.000 copy.py:243(_keep_alive)
    22470    0.043    0.000    0.220    0.000 copy.py:259(_reconstruct)
    44940    0.015    0.000    0.174    0.000 copy.py:264(<genexpr>)
      135    0.065    0.000    0.111    0.001 dag.py:166(topological_generations)
      

In [171]:
for node in L:
        for pred in G1.predecessors(node):
                predecessors[node].update(predecessors[pred])
        for original_node in C[node]:
            ancestors[original_node] = predecessors[node]

In [ ]:
'''def find_all_ancestors(G, C, G1):
    # Initialize an empty dictionary to store the ancestors of each node
    ancestors = {node: set() for node in G.nodes()}
    S = set()
    
    for node in nx.topological_sort(G1):
        node_ancestors = C[node].union(S)
        for original_node in C[node]:
            ancestors[original_node] = node_ancestors
        S.update(node_ancestors)

    # Use a list comprehension for u
    u = [ancestors[node] for node in G.nodes()]
    return u'''

In [97]:
Graph = fb_graph
C = list(nx.strongly_connected_components(Graph))
G1 = nx.condensation(Graph, C)

In [26]:
# Initialize a list to store the ancestors of each node
ancestors = [[] for _ in range(len(Graph.nodes()))]
predecessors = {node: set() for node in G1.nodes()}

for node in nx.topological_sort(G1):
    for pred in G1.predecessors(node):
        predecessors[node] = predecessors[node].union(C[pred])

    components = list(C[node])
    for original_node in C[node]:
        og_predecessors = [supernode for supernode in predecessors[node]]
        ancestors[original_node] = components + og_predecessors

u = [ancestors[node] for node in Graph.nodes()]




In [98]:
networkx.topological_generations(G1)

<generator object topological_generations at 0x1253b8cf0>